<h1>Comparison of Transfer Learning Models - Frontal View</h1>

<hr>

<p>Continuing with the model selection process, we proceed to train the three models demonstrating the highest accuracy from the previous step over a span of 10 epochs. It's important to note that, at present, we are not employing any form of data augmentations.</p>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import inspect
import glob
import os
import matplotlib.image as mpimg
import numpy as np
import seaborn as sns
import pandas as pd
import inspect
from tqdm import tqdm
from keras.preprocessing import image
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
from google.colab import drive

from sklearn.model_selection import StratifiedKFold

In [ ]:
from keras.preprocessing import image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
batch_size = 32
data_path = "/content/drive/MyDrive/sve_F-bez augumentacije/Train"
n_folds =5
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
all_files = glob.glob(os.path.join(data_path, '*/*'))
labels = [os.path.basename(os.path.dirname(fp)) for fp in all_files]

In [ ]:
# List all available models
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}

In [ ]:
len(model_dictionary)

71

In [ ]:
new_model_dict = {key: model_dictionary[key] for key in model_dictionary  if key in ['Xception', 'InceptionV3','ResNet50V2']}

In [ ]:
len(new_model_dict)

3

In [ ]:
def preprocess_data_for_fold(train_files, val_files, img_size):
    datagen = image.ImageDataGenerator(rescale=1./255.)

    # Extracting the labels from the directory structure
    train_labels = [os.path.basename(os.path.dirname(fp)) for fp in train_files]
    val_labels = [os.path.basename(os.path.dirname(fp)) for fp in val_files]

    train_gen = datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': train_files, 'class': train_labels}),
        directory=None,
        x_col="filename",
        y_col="class",
        class_mode="binary",
        target_size=img_size,
        batch_size=batch_size,
        shuffle=True
    )

    val_gen = datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': val_files, 'class': val_labels}),
        directory=None,
        x_col="filename",
        y_col="class",
        class_mode="binary",
        target_size=img_size,
        batch_size=batch_size,
        shuffle=False
    )

    return train_gen, val_gen


In [ ]:
def test_preprocces(img_size):
  test_datagen = image.ImageDataGenerator(rescale=1./255.)
  test_generator = test_datagen.flow_from_directory("/content/drive/MyDrive/sve_F-bez augumentacije/Test",
                                        target_size=img_size,
                                        class_mode='binary',
                                        batch_size=batch_size,
                                        shuffle=False)
  return test_generator

In [ ]:
def build_model(model_1,input_shape):
    pre_trained_model = model_1(include_top=False, pooling='avg', input_shape=input_shape)
    pre_trained_model.trainable = False

    model = tf.keras.models.Sequential()
    model.add(pre_trained_model)
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Loop over each model available in Keras
model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_accuracy': [], 'test_accuracy': []}
for model_name, model_1 in tqdm(list(new_model_dict.items())):
    # Special handling for "NASNetLarge" since it requires input images with size (331,331)

    fold_no = 1
    all_acc = []
    all_loss = []

    for train_indices, val_indices in kfold.split(all_files,labels):
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

    # Preprocess data and build model (assuming these functions are defined)
        train_files = [all_files[i] for i in train_indices]
        val_files = [all_files[i] for i in val_indices]
        if 'NASNetLarge' in model_name:
          img_size =(331,331)
          input_shape =(331,331,3)
          train_gen, val_gen = preprocess_data_for_fold(train_files, val_files, img_size)
        elif 'InceptionResNetV2' or 'InceptionV3' or 'Xception' in model_name:
          img_size=(299,299)
          input_shape=(299,299,3)
          train_gen, val_gen = preprocess_data_for_fold(train_files, val_files, img_size)
        else:
          img_size=(224,224)
          input_shape=(224,224,3)
          train_gen, val_gen = preprocess_data_for_fold(train_files, val_files, img_size)


        model = build_model(model_1,input_shape)

    # Fit data to model
        history = model.fit(train_gen, epochs=10, validation_data=val_gen)

    # Evaluate the model on validation set
        scores = model.evaluate(val_gen, verbose=0)
        print(f'Score for fold {fold_no}: Loss of {scores[0]}; Accuracy of {round(scores[1]*100, 2)}%')
        all_acc.append(scores[1] * 100)
        all_loss.append(scores[0])

    # Increase fold number
        fold_no += 1

# Provide average scores
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(len(all_acc)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {all_loss[i]} - Accuracy: {all_acc[i]}%')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    print(f'> Accuracy: {np.mean(all_acc)} (+- {np.std(all_acc)})')
    print(f'> Loss: {np.mean(all_loss)}')
    print('------------------------------------------------------------------------')
    if 'NASNetLarge' in model_name:
        test_processed = test_preprocces(img_size)
    elif 'InceptionResNetV2' or 'InceptionV3' or 'Xception' in model_name:
        test_processed = test_preprocces(img_size)
    else:
        test_processed = test_preprocces(img_size)

    test_acc = model.evaluate(test_processed)
    predictions = model.predict(test_processed)
    predicted_classes = np.where(predictions < 0.5, 0, 1)
    true_classes = test_processed.classes
    class_labels = list(test_processed.class_indices.keys())

    report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
    print(report)

    # Calculate all relevant metrics
    model_benchmarks['model_name'].append(model_name)
    model_benchmarks['num_model_params'].append(model.count_params())
    model_benchmarks['validation_accuracy'].append(np.mean(all_acc))
    model_benchmarks['test_accuracy'].append(test_acc[-1])


  0%|          | 0/3 [00:00<?, ?it/s]

------------------------------------------------------------------------
Training for fold 1 ...
Found 125 validated image filenames belonging to 2 classes.
Found 32 validated image filenames belonging to 2 classes.
87910968/87910968 [==============================] - 1s 0us/step
Epoch 1/10
4/4 [==============================] - 43s 9s/step - loss: 0.7037 - accuracy: 0.5360 - val_loss: 0.7065 - val_accuracy: 0.5000
Epoch 2/10
4/4 [==============================] - 2s 474ms/step - loss: 0.6926 - accuracy: 0.5360 - val_loss: 0.6377 - val_accuracy: 0.6250
Epoch 3/10
4/4 [==============================] - 2s 640ms/step - loss: 0.6470 - accuracy: 0.6240 - val_loss: 0.6069 - val_accuracy: 0.6875
Epoch 4/10
4/4 [==============================] - 2s 612ms/step - loss: 0.6158 - accuracy: 0.6880 - val_loss: 0.5801 - val_accuracy: 0.8438
Epoch 5/10
4/4 [==============================] - 2s 459ms/step - loss: 0.5836 - accuracy: 0.7680 - val_loss: 0.5544 - val_accuracy: 0.7812
Epoch 6/10
4/4 [=====

 33%|███▎      | 1/3 [04:26<08:53, 266.79s/it]

              precision    recall  f1-score   support

           F       0.66      0.76      0.70        33
           M       0.72      0.62      0.67        34

    accuracy                           0.69        67
   macro avg       0.69      0.69      0.69        67
weighted avg       0.69      0.69      0.69        67

------------------------------------------------------------------------
Training for fold 1 ...
Found 125 validated image filenames belonging to 2 classes.
Found 32 validated image filenames belonging to 2 classes.
94668760/94668760 [==============================] - 1s 0us/step
Epoch 1/10
4/4 [==============================] - 14s 2s/step - loss: 0.7496 - accuracy: 0.4720 - val_loss: 0.7546 - val_accuracy: 0.4688
Epoch 2/10
4/4 [==============================] - 2s 484ms/step - loss: 0.7058 - accuracy: 0.5040 - val_loss: 0.6889 - val_accuracy: 0.5312
Epoch 3/10
4/4 [==============================] - 2s 502ms/step - loss: 0.6549 - accuracy: 0.6320 - val_loss: 0.64

 67%|██████▋   | 2/3 [07:30<03:37, 217.82s/it]

              precision    recall  f1-score   support

           F       0.76      0.76      0.76        33
           M       0.76      0.76      0.76        34

    accuracy                           0.76        67
   macro avg       0.76      0.76      0.76        67
weighted avg       0.76      0.76      0.76        67

------------------------------------------------------------------------
Training for fold 1 ...
Found 125 validated image filenames belonging to 2 classes.
Found 32 validated image filenames belonging to 2 classes.
83683744/83683744 [==============================] - 1s 0us/step
Epoch 1/10
4/4 [==============================] - 18s 3s/step - loss: 0.6905 - accuracy: 0.5440 - val_loss: 0.6624 - val_accuracy: 0.5625
Epoch 2/10
4/4 [==============================] - 2s 584ms/step - loss: 0.6650 - accuracy: 0.6320 - val_loss: 0.6237 - val_accuracy: 0.8750
Epoch 3/10
4/4 [==============================] - 2s 513ms/step - loss: 0.6444 - accuracy: 0.7040 - val_loss: 0.59

100%|██████████| 3/3 [10:41<00:00, 213.75s/it]

              precision    recall  f1-score   support

           F       0.77      0.73      0.75        33
           M       0.75      0.79      0.77        34

    accuracy                           0.76        67
   macro avg       0.76      0.76      0.76        67
weighted avg       0.76      0.76      0.76        67



In [ ]:
# Convert Results to DataFrame for easy viewing
benchmark_df = pd.DataFrame(model_benchmarks)
benchmark_df.sort_values('num_model_params', inplace=True) # sort in ascending order of num_model_params column
benchmark_df

,model_name,num_model_params,validation_accuracy,test_accuracy
2,Xception,20863529,72.520161,0.761194
0,InceptionV3,21804833,76.975807,0.686567
1,ResNet50V2,23566849,76.411290,0.761194
